In [1]:
import numpy as np
import pickle
# from Utils.TimeLogger import log
from scipy.sparse import csr_matrix
import time
import scipy.sparse as sp
def ok(year):
	return True
	if year >= 2016 and year <= 2019:
		return True
minn = 1647180684
maxx = 0
def transTime(date):
	timeArr = time.strptime(date, '%Y-%m-%dT%H:%M:%SZ')
	year = time.mktime(timeArr)
	global minn
	global maxx
	minn = min(minn, year)
	maxx = max(maxx, year)
	if ok(year):
		return year
	return None

In [2]:
def mapping(infile):
    
	usrId = dict()
	itmId = dict()
	usrid, itmid = [0, 0]
	interaction = list()
	with open(infile, 'r', encoding='utf-8') as fs:
		for line in fs:
			arr = line.strip().split()
			# print(arr)
			row = int(arr[0])
			col = int(arr[-1])
			timeStamp = transTime(arr[1])
			if timeStamp is None:
				continue
			if row not in usrId:
				usrId[row] = usrid
				interaction.append(dict())
				usrid += 1
			if col not in itmId:
				itmId[col] = itmid
				itmid += 1
			usr = usrId[row]
			itm = itmId[col]
			if(itm not in interaction[usr]):
				interaction[usr][itm]=[]
			interaction[usr][itm].extend([timeStamp])
			# print(interaction)
	print(minn, maxx)
	return interaction, usrid, itmid

In [4]:
print(minn,maxx)

1647180684 0


In [3]:
def checkFunc1(cnt):
	return cnt >= 10
def checkFunc2(cnt):
	return cnt >= 5
def checkFunc3(cnt):
	return cnt >= 5

def filter(interaction, usrnum, itmnum, ucheckFunc, icheckFunc, filterItem=True):
    # get keep set
    usrKeep = set()
    itmKeep = set()
    itmCnt = [0] * itmnum
    for usr in range(usrnum):
        data = interaction[usr]
        usrCnt = 0
        for col in data:
            itmCnt[col] += 1
            usrCnt += 1
        if ucheckFunc(usrCnt):
            usrKeep.add(usr)
    for itm in range(itmnum):
        if not filterItem or icheckFunc(itmCnt[itm]):
            itmKeep.add(itm)

    # filter data
    retint = list()
    usrid = 0
    itmid = 0
    itmId = dict()
    user_id = list()
    item_id = list()
    time_int = list()
    for row in range(usrnum):
        if row not in usrKeep:
            continue
        usr = usrid
        usrid += 1
        retint.append(dict())
        data = interaction[row]
        for col in data:
            if col not in itmKeep:
                continue
            if col not in itmId:
                itmId[col] = itmid
                itmid += 1
            itm = itmId[col]
            retint[usr][itm] = data[col]
            for time_div in data[col]:
                user_id.append(usr+1)
                item_id.append(itm+1)
                time_int.append(int(time_div))
    return retint, usrid, itmid, user_id, item_id, time_int

In [4]:
def negSamp(temLabel, sampSize, nodeNum):
    negset = [None] * sampSize
    cur = 0
    while cur < sampSize:
        rdmItm = np.random.choice(nodeNum)
        if rdmItm not in temLabel:
            negset[cur] = rdmItm+1
            cur += 1
    return negset

def split(interaction, usrnum, itmnum):
    np.random.seed(100)
    pickNum = 10000
    # random pick
    usrPerm = np.random.permutation(usrnum)
    pickUsr = usrPerm[:pickNum]

    tstInt = [None] * usrnum
    trainPos = [None]* usrnum
    exception = 0
    user_id=list()
    item_id=list()
    time_id=list()
    neg_items=list()
    for usr in pickUsr:
        temp = list()
        data = interaction[usr]
        for itm in data:
            if (itm is not None and data[itm] is not None):
                temp.append((itm, data[itm]))
        if len(temp) == 0:
            print(usr)
            exception += 1
            continue
        # print('1',temp)
        temp.sort(key=lambda x: x[1])
        tstInt[usr] = temp[-1][0]
        # user_id.append(usr+1)
        # item_id.append(tstInt[usr]+1)
        # time_id.append(int(interaction[usr][tstInt[usr]][-1]))
        # neg_items.append(negSamp(interaction[usr],99,itmnum))
        interaction[usr][tstInt[usr]] = None
    user_behavior_sequence=list()
    for usr in range(usrnum):
        temp = list()
        data = interaction[usr]
        for itm in data:
            if(itm is not None and data[itm] is not None):
                temp.append((itm, data[itm]))
        if len(temp) == 0:
            print('2',usr)
            exception += 1
            continue
        #print(temp)
        temp.sort(key=lambda x: x[1])
        user_behavior_sequence.append(np.array(temp)[:-1,0])
        trainPos[usr] = temp[-1][0]
        interaction[usr][temp[-1][0]] = None
    print(user_id,time_id,time_id,neg_items)
    # train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_id, 'neg_items':neg_items})
    # train.to_csv('test_yelp_4.csv',index=False,sep='\t')
    # print(train)
    print('Exception:', exception, np.sum(np.array(tstInt)!=None))
    return interaction, tstInt, trainPos, user_behavior_sequence

In [5]:
from pandas.core.frame import DataFrame
prefix = 'D:/edgedownload/'
print('Start')
minn = 1647180684
maxx = 0
interaction, usrnum, itmnum = mapping(prefix + 'Gowalla_totalCheckins.txt')
print('Id Mapped, usr %d, itm %d' % (usrnum, itmnum))

Start
1233695858.0 1287782526.0
Id Mapped, usr 107092, itm 1280969


In [6]:
checkFuncs = [checkFunc1, checkFunc2, checkFunc3]
for i in range(3):
    filterItem = True if i < 1 else False
    interaction, usrnum, itmnum, user_id, item_id, time_int = filter(interaction, usrnum, itmnum, checkFuncs[i], checkFuncs[i], filterItem)
    print('Filter', i, 'times:', usrnum, itmnum)
    '''
    if(i==2):
        train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_int})
        train.to_csv('train_gowalla.csv',index=False,sep='\t')
        print(train)
    '''
print('Sparse Samples Filtered, usr %d, itm %d' % (usrnum, itmnum))


Filter 0 times: 68709 57445
Filter 1 times: 50821 57440
Filter 2 times: 50821 57440
Sparse Samples Filtered, usr 50821, itm 57440


In [17]:
ret=count_frequency(interaction,usrnum,itmnum)

[  66 1864 1562 ...    5   20   26]


In [15]:
def count_frequency(interaction,usernum,itmnum):
    count_item=np.zeros([itmnum],dtype=int)
    for i in range(itmnum):
        for j in range(usernum):
            if(i in interaction[j]):
                count_item[i]+=len(interaction[j][i])
    print(count_item)
    return np.argsort(count_item)
    
    
    

In [18]:
neg_frequency=ret

[29224 57406 57337 ...    61    62  8944]


In [42]:
with open(prefix+'gowalla_sort', 'wb') as fs:
	pickle.dump(neg_frequency, fs)

In [35]:
def negSamp_fre(temLabel, sampSize, nodeNum):
    negset = [None] * sampSize
    cur = 0
    i = 0
    while cur < sampSize:
        rdmItm = neg_frequency[-i]# np.random.choice(nodeNum)
        if rdmItm not in temLabel:
            negset[cur] = rdmItm+1
            cur += 1
        i += 1
    return negset

def split(interaction, usrnum, itmnum):
    np.random.seed(100)
    pickNum = 10000
    # random pick
    usrPerm = np.random.permutation(usrnum)
    pickUsr = usrPerm[:pickNum]

    tstInt = [None] * usrnum
    exception = 0
    user_id=list()
    item_id=list()
    time_id=list()
    neg_items=list()
    for usr in pickUsr:
        temp = list()
        data = interaction[usr]
        for itm in data:
            temp.append((itm, data[itm]))
        if len(temp) == 0:
            exception += 1
            continue
        temp.sort(key=lambda x: x[1])
        tstInt[usr] = temp[-1][0]
        user_id.append(usr+1)
        item_id.append(tstInt[usr]+1)
        time_id.append(int(interaction[usr][tstInt[usr]][-1]))
        neg_items.append(negSamp_fre(interaction[usr],99,itmnum))
        interaction[usr][tstInt[usr]] = None
    print(user_id,time_id,time_id,neg_items)
    train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_id, 'neg_items':neg_items})
    train.to_csv('test_gowalla.csv',index=False,sep='\t')
    print(train)
    print('Exception:', exception, np.sum(np.array(tstInt)!=None))
    return interaction, tstInt

In [ ]:
#def split_by_frequency():

In [7]:
def trans_sub(interaction, usrnum, itmnum, gragh_num):
	global minn
	global maxx
	interval = (maxx-minn)/gragh_num
	rcd = [[list(), list(), list()]]
	for i in range(gragh_num-1):
		rcd.append([list(), list(), list()])
	timeMat = sp.dok_matrix((usrnum, itmnum), dtype=np.int)
	for usr in range(usrnum):
		if interaction[usr] == None:
			continue
		data = interaction[usr]
		for col in data:
			if data[col] != None:
				for one_data in data[col]:
					gragh_no=int(((one_data-minn)/interval)-1)
					# print(gragh_no,one_data)
					rcd[gragh_no][0].append(usr)
					rcd[gragh_no][1].append(usrnum+col)
					rcd[gragh_no][2].append(1.0)
					rcd[gragh_no][0].append(usrnum+col)
					rcd[gragh_no][1].append(usr)
					rcd[gragh_no][2].append(1.0)
					timeMat[usr,col]=gragh_no
	intMat=list()
	for i in range(gragh_num):
		intMat.append(csr_matrix((rcd[i][2], (rcd[i][0], rcd[i][1])), shape=(usrnum+itmnum, usrnum+itmnum)))
		# intMat.append(normalized_adj_single(csr_matrix((rcd[i][2], (rcd[i][0], rcd[i][1])), shape=(usrnum+itmnum, usrnum+itmnum))))
		print(intMat[i])
	return intMat, timeMat.tocsr()

In [8]:
print(interaction, usrnum, itmnum)
trnInt, tstInt, trainPos, user_behavior_sequence = split(interaction, usrnum, itmnum)
print('Datasets Splited')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\Users\LIU-YUXI\AppData\Local\Temp/ipykernel_28108/2224593916.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  user_behavior_sequence.append(np.array(temp)[:-1,0])


[] [] [] []
Exception: 0 10000
Datasets Splited


In [18]:
# print(trainPos)
with open(prefix+'trnPosGowalla', 'wb') as fs:
	pickle.dump(trainPos, fs)

In [11]:
print(user_behavior_sequence[0])

[116 115 114 113 112 111 110 109 108 107 106 105 104 103 102 101 100 99 98
 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74
 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50
 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26
 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1]


In [12]:
tstInt

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 595,
 626,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 7,
 1756,
 None,
 None,
 1,
 None,
 None,
 None,
 2193,
 None,
 None,
 None,
 None,
 2427,
 None,
 None,
 None,
 2475,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 3138,
 None,
 None,
 None,
 6,
 3269,
 3279,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 4054,
 None,
 None,
 3563,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 177,
 61,
 None,
 None,
 None,
 None,
 None,
 None,
 5271,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 6022,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 6502,
 None,
 None,
 None,
 5239,
 None,
 6,
 641,
 None,
 7548,
 None,
 None,
 None,
 None,
 93,
 6476,
 None,
 7883,
 None,
 None,
 None,
 None,
 None,
 None,
 3195,
 8301,
 None,
 None,
 None,
 8434,
 8495,
 8497,
 None,
 None,
 None,
 None,
 None

In [13]:
with open(prefix+'tst_int_gowalla_reversed2', 'wb') as fs:
	pickle.dump(tstInt, fs)

In [14]:
def trans(interaction, usrnum, itmnum):
	r, c, d = [list(), list(), list()]
	for usr in range(usrnum):
		if interaction[usr] == None:
			continue
		data = interaction[usr]
		for col in data:
			if data[col] != None:
				for one_data in data[col]:
					r.append(usr)
					c.append(col)
					d.append(1.0)
	intMat = csr_matrix((d, (r, c)), shape=(usrnum, itmnum))
	return intMat

In [15]:
trnMat = [trans(trnInt, usrnum, itmnum)]
subMat,timeMat=trans_sub(trnInt, usrnum, itmnum, 12)
print(timeMat)
trnMat.append(subMat)
trnMat.append(timeMat)
print('Train Mat Done')
with open(prefix+'trn_mat_time_gowalla_reversed', 'wb') as fs:
	pickle.dump(trnMat, fs)

C:\Users\LIU-YUXI\AppData\Local\Temp/ipykernel_28108/1907805393.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  timeMat = sp.dok_matrix((usrnum, itmnum), dtype=np.int)


  (9, 50874)	1.0
  (9, 51528)	1.0
  (48, 50873)	1.0
  (48, 51030)	1.0
  (48, 51303)	1.0
  (48, 53520)	1.0
  (48, 54006)	1.0
  (48, 54007)	1.0
  (48, 54008)	1.0
  (48, 54009)	1.0
  (48, 54010)	1.0
  (48, 54011)	1.0
  (48, 54012)	1.0
  (48, 54013)	2.0
  (73, 50873)	1.0
  (73, 50881)	2.0
  (73, 55017)	1.0
  (78, 55162)	3.0
  (78, 55186)	1.0
  (110, 50933)	1.0
  (110, 51029)	1.0
  (110, 51030)	1.0
  (110, 51286)	1.0
  (110, 52475)	1.0
  (110, 52476)	1.0
  :	:
  (88947, 40374)	1.0
  (89107, 3156)	1.0
  (89324, 2639)	2.0
  (89342, 4886)	2.0
  (89446, 1877)	1.0
  (89560, 3378)	1.0
  (89750, 4886)	2.0
  (90073, 3321)	1.0
  (90073, 4886)	1.0
  (90105, 38240)	1.0
  (90791, 15519)	2.0
  (91224, 15908)	1.0
  (91323, 2954)	1.0
  (91404, 3156)	1.0
  (91404, 3184)	1.0
  (91693, 2954)	1.0
  (92541, 3156)	3.0
  (92541, 3195)	1.0
  (93146, 3365)	1.0
  (94518, 40374)	1.0
  (94736, 5592)	2.0
  (94806, 3422)	1.0
  (94983, 38240)	1.0
  (95806, 3422)	1.0
  (95806, 15987)	1.0
  (18, 52475)	1.0
  (18, 52476)	1

  (0, 50822)	11.0
  (0, 50823)	1.0
  (0, 50824)	1.0
  (0, 50825)	1.0
  (0, 50826)	1.0
  (0, 50827)	6.0
  (0, 50828)	1.0
  (0, 50829)	2.0
  (0, 50830)	1.0
  (0, 50831)	1.0
  (0, 50832)	1.0
  (0, 50833)	1.0
  (0, 50834)	1.0
  (0, 50835)	1.0
  (0, 50836)	1.0
  (0, 50837)	2.0
  (0, 50838)	1.0
  (0, 50839)	1.0
  (0, 50840)	1.0
  (0, 50841)	1.0
  (0, 50842)	1.0
  (0, 50843)	1.0
  (0, 50844)	1.0
  (0, 50845)	1.0
  (0, 50846)	1.0
  :	:
  (108239, 33890)	1.0
  (108240, 33892)	2.0
  (108241, 43183)	1.0
  (108241, 43185)	4.0
  (108243, 34267)	1.0
  (108243, 43301)	1.0
  (108243, 43943)	1.0
  (108245, 37587)	1.0
  (108245, 40961)	1.0
  (108245, 42130)	3.0
  (108245, 44675)	1.0
  (108246, 35566)	1.0
  (108246, 45193)	1.0
  (108247, 35762)	1.0
  (108247, 36527)	1.0
  (108249, 35859)	1.0
  (108249, 42162)	1.0
  (108250, 42519)	3.0
  (108250, 42523)	2.0
  (108251, 35063)	1.0
  (108251, 35065)	1.0
  (108251, 48570)	1.0
  (108255, 45271)	1.0
  (108256, 45364)	2.0
  (108257, 45073)	1.0
  (115, 55146)	1.0

In [40]:
trnMat

[<50821x57440 sparse matrix of type '<class 'numpy.float64'>'
 	with 1292695 stored elements in Compressed Sparse Row format>,
 [<108261x108261 sparse matrix of type '<class 'numpy.float64'>'
  	with 1824 stored elements in Compressed Sparse Row format>,
  <108261x108261 sparse matrix of type '<class 'numpy.float64'>'
  	with 1392 stored elements in Compressed Sparse Row format>,
  <108261x108261 sparse matrix of type '<class 'numpy.float64'>'
  	with 2244 stored elements in Compressed Sparse Row format>,
  <108261x108261 sparse matrix of type '<class 'numpy.float64'>'
  	with 23696 stored elements in Compressed Sparse Row format>,
  <108261x108261 sparse matrix of type '<class 'numpy.float64'>'
  	with 105548 stored elements in Compressed Sparse Row format>,
  <108261x108261 sparse matrix of type '<class 'numpy.float64'>'
  	with 247178 stored elements in Compressed Sparse Row format>,
  <108261x108261 sparse matrix of type '<class 'numpy.float64'>'
  	with 399096 stored elements in C

In [1]:
import pandas as pd
train=pd.read_csv('train_.csv',sep='\t')
train

,user_id,item_id,time
0,1,1,1287503727
1,1,2,1287411463
2,1,2,1286883880
3,1,2,1286867943
4,1,2,1286378734
...,...,...,...
2021652,50821,21942,1284648014
2021653,50821,21942,1284571445
2021654,50821,21948,1286645754
2021655,50821,21820,1286048124


In [21]:
maxx=train.time.max()
minn=train.time.min()

In [4]:
df_clear = train.drop(train[(train['user_id']==test.user_id[0]) &(train['item_id']==test.item_id[0]) ].index)

In [17]:
df_clear[df_clear.user_id==test.user_id[0]]


,user_id,item_id,time
1612866,34960,34960,1286871673
1612867,34960,34960,1282032218
1612868,34960,9286,1286635582
1612869,34960,9286,1281805086
1612870,34960,44839,1286612235
...,...,...,...
1612947,34960,9137,1272089125
1612948,34960,15587,1272011303
1612949,34960,22871,1271247631
1612950,34960,55664,1270805475


In [20]:
len(train[(train['user_id']==test.user_id[0]) &(train['item_id']==test.item_id[0]) ])

34

In [21]:
for i in range(1,10000):
    # print(test.item_id[i],test.user_id[i])
    df_clear.drop(df_clear[(df_clear['user_id']==test.user_id[i]) &(df_clear['item_id']==test.item_id[i]) ].index,inplace=True)

In [3]:
import pandas as pd
test=pd.read_csv('test_gowalla.csv',sep='\t')
test

,user_id,item_id,time,neg_items
0,34960,37407,1270511143,"[21707, 3045, 29150, 8380, 38259, 3353, 51720,..."
1,4866,36665,1279044340,"[49937, 23024, 21461, 6393, 23386, 15020, 1885..."
2,48644,30998,1287216200,"[40437, 40666, 54129, 28265, 26282, 2769, 1408..."
3,16224,30982,1262539420,"[41626, 48431, 42024, 46613, 33287, 35693, 269..."
4,27724,11359,1274630562,"[45484, 36114, 40791, 51654, 6562, 51648, 1265..."
...,...,...,...,...
9995,33538,53289,1287291041,"[34772, 47586, 27127, 23686, 46291, 19035, 186..."
9996,17416,56463,1278885182,"[30940, 42880, 31801, 38513, 7978, 21471, 1175..."
9997,22885,18838,1265304316,"[23211, 46596, 29143, 25833, 10515, 15254, 373..."
9998,8164,30821,1287169212,"[13717, 9092, 8828, 56872, 32536, 34532, 42761..."


In [26]:
df_clear[df_clear.user_id==4866]

,user_id,item_id,time
529152,4866,3933,1265998360
529153,4866,34896,1264625094
529154,4866,4987,1264423076
529155,4866,4987,1262953642
529156,4866,13282,1263838812
529157,4866,21143,1263754681
529158,4866,1711,1263578895
529159,4866,8166,1263557967
529160,4866,8123,1263318153
529161,4866,21137,1263062928


In [27]:
train[train.user_id==4866]

,user_id,item_id,time
529151,4866,36665,1279044340
529152,4866,3933,1265998360
529153,4866,34896,1264625094
529154,4866,4987,1264423076
529155,4866,4987,1262953642
529156,4866,13282,1263838812
529157,4866,21143,1263754681
529158,4866,1711,1263578895
529159,4866,8166,1263557967
529160,4866,8123,1263318153


In [24]:
df_clear.to_csv('train_gowalla_srgnn.csv',index=False,sep='\t')

In [2]:
import pickle
train_data = pickle.load(open('D:/experiment/SR-GNN-master/datasets/yoo/train.txt', 'rb'))

In [5]:
maxxx=0
for i in range(len(train_data[0])):
    # print(len(train_data[0][i]))
    maxxx=max(maxxx,len(train_data[0][i]))
maxxx

145

In [6]:
len(train_data[0])

369859

In [8]:
train_data2 = pickle.load(open('D:/experiment/CLSR/sample/train.txt', 'rb'))

In [10]:
maxxx=0
for i in range(len(train_data2[0])):
    # print(len(train_data[0][i]))
    maxxx=max(maxxx,len(train_data2[0][i]))
maxxx

942

In [11]:
len(train_data2[0])

37705